# ScanDMM 项目学习指南

本notebook将帮助你系统化地理解ScanDMM项目，每个任务完成后，请在对应的思考区域记录你的理解和感悟。

## 学习路线图

1. **基础工具函数理解** - 坐标转换、图像处理
2. **球面卷积理解** - 360°图像的特殊处理
3. **模型组件理解** - DMM的各个模块
4. **完整模型理解** - DMM整体架构
5. **训练流程理解** - 如何训练模型
6. **推理流程理解** - 如何使用模型
7. **数据处理理解** - 数据如何准备
8. **整体串联** - 完整流程理解

## 使用说明

- 每个任务都有明确的目标和时间估算
- 阅读对应的源代码文件
- 运行代码示例加深理解
- 在思考区域记录你的想法
- 按顺序完成，不要跳步


---

# 任务1: 理解基础工具函数（suppor_lib.py）

**目标**: 理解360°图像处理中的坐标系统转换

**时间**: 30-45分钟

**关键概念**:
- 球面坐标 (lat, lon)
- 3D单位球坐标 (x, y, z)
- 平面坐标 (归一化的x, y)
- 等距柱状投影 (Equirectangular Projection)

**需要阅读的文件**: `suppor_lib.py`


In [ ]:
# 任务1.1: 阅读并理解坐标转换函数
# 打开 suppor_lib.py，重点关注以下函数：
# 1. sphere2xyz() - 球面坐标转3D坐标
# 2. xyz2sphere() - 3D坐标转球面坐标
# 3. sphere2plane() - 球面坐标转平面坐标
# 4. xyz2plane() - 3D坐标转平面坐标

import torch
import math
import numpy as np

pi = math.pi

# 让我们手动实现并测试这些函数
def sphere2xyz_manual(sphere_cord):
    """
    输入: (lat, lon) 形状 = (n, 2)
    输出: (x, y, z) 形状 = (n, 3)
    
    将球面坐标转换为3D单位球上的点
    """
    lat, lon = sphere_cord[:, 0], sphere_cord[:, 1]
    lat_rad = lat / 180 * pi  # 纬度转弧度
    lon_rad = lon / 180 * pi  # 经度转弧度
    
    x = torch.cos(lat_rad) * torch.cos(lon_rad)
    y = torch.cos(lat_rad) * torch.sin(lon_rad)
    z = torch.sin(lat_rad)
    
    return torch.cat((x.view(-1, 1), y.view(-1, 1), z.view(-1, 1)), 1)

# 测试：创建一个测试点
test_point = torch.tensor([[0.0, 0.0]])  # 赤道，0度经度
xyz = sphere2xyz_manual(test_point)
print(f"球面坐标 (0°, 0°): {test_point}")
print(f"3D坐标: {xyz}")
print(f"验证单位球: {torch.norm(xyz)} = 1.0")

# 测试更多点
test_points = torch.tensor([
    [0.0, 0.0],      # 赤道，0度
    [90.0, 0.0],     # 北极
    [-90.0, 0.0],   # 南极
    [0.0, 90.0],     # 赤道，东经90度
    [0.0, -90.0],   # 赤道，西经90度
])

xyz_points = sphere2xyz_manual(test_points)
print("\n多个测试点的转换:")
for i, (sp, xyz) in enumerate(zip(test_points, xyz_points)):
    print(f"点{i+1}: 球面{sp.tolist()} -> 3D{xyz.tolist()}")


### 📝 任务1思考区域

**问题1**: 为什么需要这些坐标转换？360°图像和普通图像有什么不同？

**你的思考**:

---

**问题2**: 等距柱状投影的特点是什么？为什么在360°图像处理中常用？

**你的思考**:

---

**问题3**: 在眼动追踪中，为什么使用3D坐标而不是直接使用2D平面坐标？

**你的思考**:

---

**其他感悟**:


---

# 任务2: 理解CoordConv（坐标卷积）

**目标**: 理解为什么需要添加坐标信息到卷积层

**时间**: 20-30分钟

**关键概念**:
- 标准CNN的局限性
- 坐标信息的重要性
- 如何将坐标信息融入特征图

**需要阅读的文件**: `CoordConv.py`


In [ ]:
# 任务2.1: 理解CoordConv的工作原理

import torch
import torch.nn as nn

# 模拟CoordConv的效果
def visualize_coord_conv():
    """
    演示CoordConv如何添加坐标信息
    """
    # 假设输入是3通道图像 (batch=1, channels=3, height=128, width=256)
    batch_size = 1
    channels = 3
    height = 128
    width = 256
    
    # 创建示例输入
    input_tensor = torch.randn(batch_size, channels, height, width)
    
    # 创建坐标通道
    xx_channel = torch.arange(width).float().unsqueeze(0).repeat(height, 1)
    yy_channel = torch.arange(height).float().unsqueeze(1).repeat(1, width)
    
    # 归一化到[-1, 1]
    xx_channel = (xx_channel / (width - 1)) * 2 - 1
    yy_channel = (yy_channel / (height - 1)) * 2 - 1
    
    # 扩展到batch维度
    xx_channel = xx_channel.unsqueeze(0).unsqueeze(0).repeat(batch_size, 1, 1, 1)
    yy_channel = yy_channel.unsqueeze(0).unsqueeze(0).repeat(batch_size, 1, 1, 1)
    
    # 拼接：原始3通道 + x坐标 + y坐标 = 5通道
    output = torch.cat([input_tensor, xx_channel, yy_channel], dim=1)
    
    print(f"输入形状: {input_tensor.shape}")
    print(f"输出形状: {output.shape}")
    print(f"\n坐标通道范围:")
    print(f"  X坐标: [{xx_channel.min():.2f}, {xx_channel.max():.2f}]")
    print(f"  Y坐标: [{yy_channel.min():.2f}, {yy_channel.max():.2f}]")
    
    return output

result = visualize_coord_conv()


### 📝 任务2思考区域

**问题1**: 为什么在360°图像处理中，坐标信息特别重要？

**你的思考**:

---

**问题2**: 标准CNN在处理等距柱状投影图像时可能遇到什么问题？

**你的思考**:

---

**其他感悟**:


---

# 任务3: 理解球面卷积（Sphere CNN）

**目标**: 理解如何对360°图像进行卷积操作

**时间**: 45-60分钟

**关键概念**:
- 等距柱状投影的扭曲问题
- 球面几何的卷积核采样
- grid_sample的使用

**需要阅读的文件**: `sphere_cnn.py`


In [ ]:
# 任务3.1: 理解球面卷积的核心思想

print("""
等距柱状投影的扭曲问题:

1. 在等距柱状投影中，极地附近的区域被严重拉伸
2. 标准卷积在极地附近会采样到错误的区域
3. 球面卷积通过计算球面几何来正确采样

关键理解:
- 等距柱状投影在极地附近有严重扭曲
- 标准卷积在极地附近会采样到错误的区域
- 球面卷积通过计算球面几何来正确采样
""")

# 阅读 sphere_cnn.py 中的以下函数:
# 1. cal_index() - 计算卷积核采样位置
# 2. gen_filters_coordinates() - 生成所有采样位置
# 3. SphereConv2D - 球面卷积层
# 4. Sphere_CNN - 完整的球面CNN网络


In [ ]:
# 任务3.2: 理解cal_index函数（计算卷积核采样位置）

print("""
cal_index函数的核心思想:

1. 输入: 图像中的位置 (img_r, img_c) 和图像尺寸 (h, w)
2. 转换: 像素坐标 -> 球面坐标 (phi, theta)
3. 计算: 在球面上，3x3卷积核的9个采样位置
4. 转换: 球面坐标 -> 像素坐标
5. 输出: 9个采样位置的像素坐标

关键点:
- 考虑了球面几何，而不是简单的像素偏移
- 使用grid_sample进行双线性插值采样
""")

# 阅读 sphere_cnn.py 中的 cal_index 函数
# 理解它是如何计算采样位置的


### 📝 任务3思考区域

**问题1**: 为什么不能直接对360°图像使用标准卷积？

**你的思考**:

---

**问题2**: SphereConv2D中的grid_sample是如何工作的？

**你的思考**:

---

**问题3**: 球面卷积的计算复杂度如何？有什么优化方法？

**你的思考**:

---

**其他感悟**:


---

# 任务4: 理解DMM模型组件

**目标**: 理解深度马尔可夫模型的各个组件

**时间**: 60-90分钟

**关键概念**:
- 马尔可夫链
- 变分推断
- 发射器（Emitter）
- 转移函数（Transition）
- 组合器（Combiner）

**需要阅读的文件**: `models.py`


In [ ]:
# 任务4.1: 理解马尔可夫链的基本概念

print("""
深度马尔可夫模型（DMM）的基本结构:

时间序列: x_1, x_2, ..., x_T  (观测序列，即扫描路径)
隐状态:   z_1, z_2, ..., z_T  (不可观测的隐状态)

马尔可夫假设:
  p(z_t | z_{t-1}, z_{t-2}, ..., z_1) = p(z_t | z_{t-1})
  (当前隐状态只依赖于前一个隐状态)

生成过程:
  1. z_1 ~ p(z_1)                    (初始隐状态)
  2. z_t ~ p(z_t | z_{t-1})          (转移)
  3. x_t ~ p(x_t | z_t)              (发射)

在ScanDMM中:
  - x_t: 第t个时刻的注视点坐标 (3D坐标)
  - z_t: 第t个时刻的隐状态 (100维向量)
  - 图像特征也会影响转移过程
""")


In [ ]:
# 任务4.2: 理解Emitter（发射器）

import torch
import torch.nn as nn

print("""
Emitter的作用: p(x_t | z_t)

输入: z_t (隐状态，100维)
输出: x_t的分布参数 (mu, sigma)

流程:
  z_t -> Linear -> ReLU -> Linear -> Sigmoid*2-1 -> mu (归一化到[-1,1])
  mu -> ReLU -> Linear -> Softplus -> sigma

为什么使用Sigmoid*2-1?
  - 将输出归一化到[-1, 1]范围
  - 对应3D单位球坐标的范围

为什么使用Softplus?
  - 确保sigma > 0 (标准差必须为正)
""")

# 模拟Emitter的前向传播
z_dim = 100
gaze_dim = 3  # (x, y, z) 3D坐标
hidden_dim = 100

z_t = torch.randn(1, z_dim)  # 示例隐状态

# 简化的Emitter流程
lin1 = nn.Linear(z_dim, hidden_dim)
lin2 = nn.Linear(hidden_dim, gaze_dim)
lin_sig = nn.Linear(gaze_dim, gaze_dim)
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
softplus = nn.Softplus()

hidden = relu(lin1(z_t))
mu = sigmoid(lin2(hidden)) * 2 - 1  # 归一化到[-1, 1]
sigma = softplus(lin_sig(relu(mu)))

print(f"\n示例:")
print(f"输入 z_t 形状: {z_t.shape}")
print(f"输出 mu 形状: {mu.shape}, 范围: [{mu.min():.3f}, {mu.max():.3f}]")
print(f"输出 sigma 形状: {sigma.shape}, 范围: [{sigma.min():.3f}, {sigma.max():.3f}]")


In [ ]:
# 任务4.3: 理解GatedTransition（门控转移）

print("""
GatedTransition的作用: p(z_t | z_{t-1}, image_features)

输入:
  - z_{t-1}: 前一个时刻的隐状态
  - img_feature: 图像特征（从Sphere_CNN提取）

核心思想: 门控机制
  - 计算一个权重 weight (0到1之间)
  - mu = (1-weight) * identity(z_{t-1}) + weight * transformed(z_{t-1}, img)
  - 当weight接近0时，保持状态不变
  - 当weight接近1时，根据图像特征更新状态

为什么需要门控?
  - 控制状态更新的程度
  - 平衡历史信息和当前图像信息
""")

# 阅读 models.py 中的 GatedTransition 类
# 理解门控机制如何工作


In [ ]:
# 任务4.4: 理解Combiner（组合器）和变分推断

print("""
Combiner的作用: q(z_t | z_{t-1}, x_{t:T})

这是变分推断中的后验分布近似！

为什么需要变分推断?
  - 真实后验 p(z_t | x_{1:T}) 难以计算
  - 使用变分分布 q(z_t | z_{t-1}, x_{t:T}) 来近似

输入:
  - z_{t-1}: 前一个隐状态
  - h_rnn: RNN处理反向序列 x_{t:T} 得到的特征

关键:
  - 使用反向序列 x_{t:T} 来推断 z_t
  - 结合 z_{t-1} 和 RNN特征
  - 输出 q(z_t) 的分布参数 (mu, sigma)

为什么使用反向序列?
  - 未来的观测 x_{t+1}, ..., x_T 对推断 z_t 有帮助
  - 这是变分推断中的常见技巧
""")


### 📝 任务4思考区域

**问题1**: 为什么使用马尔可夫模型而不是直接使用RNN？

**你的思考**:

---

**问题2**: 门控机制在GatedTransition中的作用是什么？

**你的思考**:

---

**问题3**: 变分推断中的ELBO损失函数是什么？为什么需要KL退火？

**你的思考**:

---

**问题4**: 图像特征如何影响扫描路径的生成？

**你的思考**:

---

**其他感悟**:


---

# 任务5: 理解完整DMM模型

**目标**: 理解model()和guide()函数的完整流程

**时间**: 60-90分钟

**关键概念**:
- 生成模型（model）
- 变分分布（guide）
- Pyro的概率编程

**需要阅读的文件**: `models.py` 中的 DMM 类


In [ ]:
# 任务5.1: 理解model()函数（生成过程）

print("""
model()函数实现了生成过程: p(x_{1:T} | z_{1:T}), p(z_{1:T})

流程:
  1. 初始化 z_0（可学习参数）
  2. 提取图像特征（通过Sphere_CNN）
  3. 对每个时间步 t = 1, 2, ..., T:
     a. 计算转移: z_mu, z_sigma = trans(z_{t-1}, img_features)
     b. 采样隐状态: z_t ~ Normal(z_mu, z_sigma)
     c. 计算发射: x_mu, x_sigma = emitter(z_t)
     d. 采样观测: x_t ~ Normal(x_mu, x_sigma)
     e. 更新: z_{t-1} = z_t

关键点:
  - 使用pyro.sample()进行采样
  - 使用mask处理变长序列
  - 训练时使用obs参数（观测到的真实值）
  - 预测时不使用obs参数（生成新样本）
""")

# 仔细阅读 models.py 中 DMM.model() 函数
# 理解每个步骤的作用


In [ ]:
# 任务5.2: 理解guide()函数（变分推断）

print("""
guide()函数实现了变分分布: q(z_{1:T} | x_{1:T})

流程:
  1. 初始化 h_0（RNN初始状态）
  2. 使用RNN处理反向序列 x_{T}, x_{T-1}, ..., x_1
  3. 将RNN输出反转回正常顺序
  4. 初始化 z_0
  5. 对每个时间步 t = 1, 2, ..., T:
     a. 组合: z_mu, z_sigma = combiner(z_{t-1}, rnn_output[t-1])
     b. 采样: z_t ~ Normal(z_mu, z_sigma)
     c. 更新: z_{t-1} = z_t

关键点:
  - RNN处理反向序列以利用未来信息
  - combiner结合历史隐状态和RNN特征
  - 这是变分推断中的后验近似
""")

# 仔细阅读 models.py 中 DMM.guide() 函数
# 理解变分推断的过程


In [ ]:
# 任务5.3: 理解Pyro的概率编程

print("""
Pyro是一个概率编程库，允许我们:

1. 定义概率模型（使用pyro.sample）
2. 定义变分分布（guide函数）
3. 自动计算ELBO损失
4. 进行变分推断

关键函数:
  - pyro.sample(name, dist, obs=...): 采样或观测
  - pyro.module(): 注册PyTorch模块
  - pyro.plate(): 批次处理
  - pyro.markov(): 标记马尔可夫依赖

ELBO (Evidence Lower BOund):
  ELBO = E_q[log p(x|z)] - KL(q(z) || p(z))
  
  第一项: 重构误差（观测的似然）
  第二项: KL散度（正则化项）
""")

# 理解Pyro如何自动计算梯度
# 理解SVI（随机变分推断）的工作原理


### 📝 任务5思考区域

**问题1**: model()和guide()中的pyro.sample()有什么不同？

**你的思考**:

---

**问题2**: 为什么在guide()中使用反向序列？

**你的思考**:

---

**问题3**: mask的作用是什么？如何处理变长序列？

**你的思考**:

---

**问题4**: 训练和预测时，model()函数的行为有什么不同？

**你的思考**:

---

**其他感悟**:


---

# 任务6: 理解训练流程（train.py）

**目标**: 理解如何训练DMM模型

**时间**: 45-60分钟

**关键概念**:
- SVI（随机变分推断）
- 批次处理
- KL退火
- 学习率衰减

**需要阅读的文件**: `train.py`


In [ ]:
# 任务6.1: 理解训练数据准备

print("""
prepare_train_data()函数的作用:

输入: train_package字典，包含:
  - 'train': 训练数据字典
    - 每个图像: {'image': Tensor, 'scanpaths': Tensor}
  - 'info': 数据集信息

输出: 整理后的数据字典
  - 'sequences': 所有扫描路径 (N, T, 3)
  - 'sequence_lengths': 每个序列的长度 (N,)
  - 'images': 对应的图像 (N, 3, H, W)
  - 'image_index': 图像索引 (N,)

关键点:
  - 将所有扫描路径展平成一维数组
  - 每个扫描路径对应一个图像
  - 序列长度固定（30秒）
""")

# 阅读 train.py 中的 prepare_train_data() 函数


In [ ]:
# 任务6.2: 理解批次处理和序列打包

print("""
get_mini_batch()函数的作用:

1. 选择一批序列
2. 按长度排序（从长到短）
3. 反转序列（用于RNN处理）
4. 创建mask（标记有效位置）
5. 使用pack_padded_sequence打包

为什么排序?
  - RNN处理变长序列时需要
  - 减少padding的浪费

为什么反转?
  - guide()中使用反向序列
  - 利用未来信息推断当前隐状态

pack_padded_sequence的作用:
  - 高效处理变长序列
  - 避免在padding位置计算
""")

# 阅读 train.py 中的 get_mini_batch() 函数


In [ ]:
# 任务6.3: 理解KL退火

print("""
KL退火（Annealing）的作用:

问题: 训练初期，KL散度项可能过大，导致:
  - 后验分布q(z)被强制接近先验p(z)
  - 模型无法学习有意义的表示

解决: 逐渐增加KL项的权重
  - 训练初期: annealing_factor较小（如0.2）
  - 训练后期: annealing_factor=1.0

公式:
  annealing_factor = min_af + (1 - min_af) * progress
  
  其中progress从0逐渐增加到1

在代码中:
  - annealing_epochs = 10（前10个epoch使用退火）
  - minimum_annealing_factor = 0.2
""")

# 阅读 train.py 中的 process_minibatch() 函数
# 理解KL退火如何实现


### 📝 任务6思考区域

**问题1**: 为什么需要pack_padded_sequence？

**你的思考**:

---

**问题2**: KL退火为什么重要？如果不用会怎样？

**你的思考**:

---

**问题3**: 学习率衰减的策略是什么？为什么选择0.99998？

**你的思考**:

---

**问题4**: 训练过程中如何监控模型性能？

**你的思考**:

---

**其他感悟**:


---

# 任务7: 理解推理流程（inference.py）

**目标**: 理解如何使用训练好的模型生成扫描路径

**时间**: 30-45分钟

**关键概念**:
- 采样生成
- 起点选择
- 多路径生成

**需要阅读的文件**: `inference.py`


In [ ]:
# 任务7.1: 理解起点采样

print("""
create_random_starting_points()函数的作用:

生成扫描路径的起始点

策略: 赤道偏置采样
  - Y坐标（纬度）: 从N(0, 0.2)采样，限制在[-1, 1]
  - X坐标（经度）: 从U(-1, 1)均匀采样

为什么赤道偏置?
  - 人们在观看360°图像时，通常从水平方向开始
  - 符合人类视觉习惯

转换:
  球面坐标 -> 3D坐标 -> 作为初始输入
""")

# 阅读 inference.py 中的 create_random_starting_points() 函数


In [ ]:
# 任务7.2: 理解预测过程

print("""
predict()函数的工作流程:

1. 对每张输入图像:
   a. 加载并预处理图像
   b. 生成多个随机起点
   c. 使用Predictive进行采样:
      - 调用model()函数（predict=True）
      - 不提供obs参数，让模型生成新样本
   d. 整理输出:
      - 3D坐标 -> 平面坐标
      - 归一化到[0, 1]
   e. 保存结果（.npy文件）
   f. 可选：可视化扫描路径

关键点:
  - num_samples控制内存使用
  - 可以生成多条不同的扫描路径
  - 每条路径都是随机采样得到的
""")

# 阅读 inference.py 中的 predict() 和 summary() 函数


### 📝 任务7思考区域

**问题1**: 为什么可以生成多条不同的扫描路径？

**你的思考**:

---

**问题2**: 起点选择对最终结果有什么影响？

**你的思考**:

---

**问题3**: 如何评估生成的扫描路径的质量？

**你的思考**:

---

**其他感悟**:


---

# 任务8: 理解数据处理（data_process.py）

**目标**: 理解如何从原始眼动数据准备训练数据

**时间**: 45-60分钟

**关键概念**:
- 眼动数据格式
- 数据增强（旋转）
- 无效数据处理

**需要阅读的文件**: `data_process.py`


In [ ]:
# 任务8.1: 理解眼动数据采样

print("""
sample_gaze_points()函数的作用:

将原始眼动数据转换为固定长度的扫描路径

流程:
  1. 将原始数据分成30个时间bin（每个1秒）
  2. 对每个bin:
     - 过滤无效点（坐标为0的点）
     - 如果bin为空，标记为(-999, -999)
     - 否则，采样第一个有效点
  3. 转换为球面坐标（减去[90, 180]偏移）

为什么采样第一个点?
  - 简化处理
  - 每个bin代表1秒的注视

数据格式:
  - 输入: 原始眼动数据 (N, 2) - (lat, lon)
  - 输出: 30个注视点 (30, 2)
""")

# 阅读 data_process.py 中的 sample_gaze_points() 函数


In [ ]:
# 任务8.2: 理解无效数据处理和数据增强

print("""
handle_empty()函数的作用:

处理扫描路径中的无效注视点

策略:
  1. 第一个点无效:
     - 如果下一个点有效，用下一个点填充
     - 否则，丢弃整个扫描路径
  
  2. 最后一个点无效:
     - 用前一个点填充
  
  3. 中间点无效:
     - 如果前后点都有效，进行插值
     - 插值时考虑球面几何（经度可能跨越180度）
     - 如果前后点无效，丢弃扫描路径

数据增强策略: 旋转360°图像
  - 对每张原始图像，旋转6个角度（每60度）
  - 同时旋转对应的扫描路径
  - 这样可以将1张图像变成6张训练样本
""")

# 阅读 data_process.py 中的 handle_empty() 和 rotate() 函数


### 📝 任务8思考区域

**问题1**: 为什么选择30秒作为扫描路径长度？

**你的思考**:

---

**问题2**: 数据增强还有哪些其他方法？

**你的思考**:

---

**问题3**: 如何处理不同长度的扫描路径？

**你的思考**:

---

**其他感悟**:


---

# 任务9: 整体串联理解

**目标**: 将各个部分串联起来，理解完整流程

**时间**: 60-90分钟

**关键概念**:
- 端到端流程
- 数据流
- 模型架构图


In [ ]:
# 任务9.1: 绘制完整的数据流图

print("""
完整的数据流:

【训练阶段】
1. 原始数据 -> data_process.py -> 训练数据(.pkl)
2. 训练数据 -> train.py -> 训练模型
   - 图像 -> Sphere_CNN -> 图像特征
   - 扫描路径 -> DMM model/guide -> ELBO损失
   - 反向传播 -> 更新参数
3. 保存模型权重

【推理阶段】
1. 输入图像 -> inference.py
2. 图像 -> Sphere_CNN -> 图像特征
3. 随机起点 + 图像特征 -> DMM model -> 生成扫描路径
4. 扫描路径 -> 坐标转换 -> 保存/可视化

【关键组件】
- Sphere_CNN: 提取360°图像特征
- DMM: 生成扫描路径序列
- 坐标转换: 在不同表示间转换
""")

# 尝试画出完整的数据流图
# 理解每个组件如何连接


In [ ]:
# 任务9.2: 理解模型的关键设计决策

print("""
关键设计决策总结:

1. 为什么使用DMM而不是RNN?
   - 显式建模隐状态
   - 可以生成多条不同的路径
   - 更好的概率解释

2. 为什么使用球面卷积?
   - 等距柱状投影有扭曲
   - 标准卷积在极地附近失效
   - 球面卷积考虑几何结构

3. 为什么图像特征影响转移?
   - 扫描路径依赖于图像内容
   - 门控机制平衡历史和图像信息

4. 为什么使用变分推断?
   - 真实后验难以计算
   - 变分推断提供可训练的近似
   - 使用反向序列利用未来信息
""")


### 📝 任务9思考区域

**问题1**: 整个系统的瓶颈在哪里？如何优化？

**你的思考**:

---

**问题2**: 如果要改进模型，你会从哪些方面入手？

**你的思考**:

---

**问题3**: 这个模型可以应用到哪些其他场景？

**你的思考**:

---

**问题4**: 你对整个项目的整体评价是什么？

**你的思考**:

---

**最终总结**:

（记录你完成所有任务后的整体理解和感悟）


---

# 扩展学习任务（可选）

如果你已经完成了所有基础任务，可以尝试以下扩展：

1. **代码实现练习**: 尝试自己实现某个组件（如Emitter）
2. **实验分析**: 修改超参数，观察效果变化
3. **可视化分析**: 可视化中间特征和隐状态
4. **性能分析**: 分析模型的训练和推理速度
5. **对比研究**: 与其他扫描路径预测方法对比

## 学习进度追踪

- [ ] 任务1: 基础工具函数理解
- [ ] 任务2: CoordConv理解
- [ ] 任务3: 球面卷积理解
- [ ] 任务4: DMM模型组件理解
- [ ] 任务5: 完整DMM模型理解
- [ ] 任务6: 训练流程理解
- [ ] 任务7: 推理流程理解
- [ ] 任务8: 数据处理理解
- [ ] 任务9: 整体串联理解

**完成日期**: ___________

**总耗时**: ___________

**主要收获**: 
